In [ ]:
!pip3 install openpyxl

In [ ]:
import pandas as pd

df = pd.read_excel(
    "./international-teaching-week-2023/data/dataset_sugar_tax.xlsx", sheet_name="Data"
)
df

In [ ]:
df[df["type"] == "TEA"].boxplot(column="price_per_oz", by="product_id")

In [ ]:
df["taxed"] = df["taxed"].map({0: "not taxed", 1: "taxed"})
df["supp"] = df["supp"].map({0: "Standard", 1: "Supplemental"})
df["store_type_str"] = df["store_type"].map(
    {1: "Large Supermarket", 2: "Small Supermarket", 3: "Pharmacy", 4: "Gas Station"}
)
df["time"] = df["time"].replace({"MAR2015": "MAR2016"}).astype("category")
df["store_id"] = df["store_id"].astype("category")
df

In [ ]:
num_stores = df["store_id"].nunique()
num_products = df["product_id"].nunique()

print(f"Number of unique stores: {num_stores}")
print(f"Number of unique products: {num_products}")

In [ ]:
freq_table_store_type = df.pivot_table(
    index=["store_type_str"],
    columns="time",
    values="price",
    aggfunc="count",
    margins=True,
)
freq_table_store_type = freq_table_store_type.reindex(
    ["Large Supermarket", "Small Supermarket", "Pharmacy", "Gas Station", "All"],
    level=0,
)

print(freq_table_store_type)

In [ ]:
freq_table_taxed = df.pivot_table(
    index=["store_type_str"],
    columns=["time", "taxed"],
    values="price",
    aggfunc="count",
    margins=True,
).reindex(
    ["Large Supermarket", "Small Supermarket", "Pharmacy", "Gas Station", "All"],
    level=0,
)
print(freq_table_taxed)

In [ ]:
freq_table_products = df.pivot_table(
    index=["type"], columns="time", values="price_per_oz", aggfunc="count", margins=True
)
print(freq_table_products)

In [ ]:
fpt = df.pivot_table(
    index=["store_id", "product_id"], columns="time", values="price", aggfunc="count"
)
mask = (fpt > 0).all(axis=1)
fpt = fpt[mask]
print(fpt.head())
testdf = df.reset_index()
testdf = testdf.set_index(["store_id", "product_id"])
testdf

In [ ]:
testdf = testdf.loc[fpt.index.values]
testdf = testdf[testdf["supp"] != "Supplemental"]
testdf

In [ ]:
testdf = testdf.reset_index()
testdf

In [ ]:
testdf["product_id"].nunique()
testdf.boxplot(column="price_per_oz", by="product_id")

In [ ]:
new_pivot = testdf.pivot_table(
    index=["taxed", "store_type_str"],
    columns="time",
    values="price_per_oz",
    aggfunc="mean",
).round(3)

new_pivot

In [ ]:
new_pivot["d1"] = new_pivot["JUN2015"] - new_pivot["DEC2014"]
new_pivot["d2"] = new_pivot["MAR2016"] - new_pivot["DEC2014"]
new_pivot

In [ ]:
df123 = new_pivot
df123 = df123.reset_index()
df123 = df123.pivot(index="store_type_str", columns="taxed", values="d1")
df123.plot.bar()

In [ ]:
df123 = new_pivot
df123 = df123.reset_index()
df123 = df123.pivot(index="store_type_str", columns="taxed", values="d2")
df123.plot.bar()